In [1]:
using InteractiveUtils

In [2]:
function pw_rec(n, x::T) where {T}
    if iszero(n)
        one(T)
    elseif isodd(n)
        pw_rec(n - 1, x) * x
    else
        pw_rec(n ÷ 2, x * x)
    end
end

pw_rec (generic function with 1 method)

In [3]:
Tuple(pw_rec(n, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [4]:
function pw_rec_pe(::Val{n}, x::T) where {n,T}
    if iszero(n)
        one(T)
    elseif isodd(n)
        pw_rec_pe(Val(n - 1), x) * x
    else
        pw_rec_pe(Val(n ÷ 2), x * x)
    end
end

pw_rec_pe (generic function with 1 method)

In [5]:
Tuple(pw_rec_pe(Val(n), "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [6]:
@code_typed pw_rec_pe(Val(2), 10)

CodeInfo(
1 ─      nothing::Nothing
│        nothing::Nothing
│   %3 = Base.mul_int(x, x)::Int64
│   %4 = Base.mul_int(1, %3)::Int64
└──      return %4
) => Int64

In [7]:
function pw_tail(n, r, x)
    if iszero(n)
        r
    elseif isodd(n)
        pw_tail(n - 1, r * x, x)
    else
        pw_tail(n ÷ 2, r, x * x)
    end
end

pw_tail (generic function with 1 method)

In [8]:
pw_tail(n, x::T) where {T} = pw_tail(n, one(T), x)

pw_tail (generic function with 2 methods)

In [9]:
Tuple(pw_tail(n, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [10]:
function pw_tail_pe(::Val{n}, r, x) where {n}
    if iszero(n)
        r
    elseif isodd(n)
        pw_tail_pe(Val(n - 1), r * x, x)
    else
        pw_tail_pe(Val(n ÷ 2), r, x * x)
    end
end

pw_tail_pe (generic function with 1 method)

In [11]:
pw_tail_pe(::Val{n}, x::T) where {n,T} = pw_tail_pe(Val(n), one(T), x)

pw_tail_pe (generic function with 2 methods)

In [12]:
Tuple(pw_tail_pe(Val(n), "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [13]:
@code_typed pw_tail_pe(Val(5), 10)

CodeInfo(
1 ─ %1 = Base.mul_int(1, x)::Int64
│   %2 = Base.mul_int(x, x)::Int64
│   %3 = Base.mul_int(%2, %2)::Int64
│   %4 = Base.mul_int(%1, %3)::Int64
└──      return %4
) => Int64

In [14]:
function pw_tail_pe_opt(::Val{n}, ::Val{r_one}, r, x) where {n,r_one}
    if iszero(n)
        r
    elseif isodd(n)
        pw_tail_pe_opt(Val(n - 1), Val(false), r_one ? x : r * x, x)
    else
        pw_tail_pe_opt(Val(n ÷ 2), Val(r_one), r, x * x)
    end
end

pw_tail_pe_opt (generic function with 1 method)

In [15]:
pw_tail_pe_opt(::Val{n}, x::T) where {n,T} =
    pw_tail_pe_opt(Val(n), Val(true), one(T), x)

pw_tail_pe_opt (generic function with 2 methods)

In [16]:
Tuple(pw_tail_pe_opt(Val(n), "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [17]:
@code_typed pw_tail_pe_opt(Val(5), 10)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.mul_int(%1, %1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
└──      return %3
) => Int64

A what about returning abstract values from functions?

In [37]:
function pw_rec_pe_abs(::Val{n}, x::T) where {n,T}
    if iszero(n)
        (Val(true), one(T))
    elseif isodd(n)
        (r_one, r) = pw_rec_pe_abs(Val(n - 1), x)
        (Val(false), r_one isa Val{true} ? x : r * x)
    else
        pw_rec_pe_abs(Val(n ÷ 2), x * x)
    end
end

pw_rec_pe_abs (generic function with 1 method)

In [38]:
Tuple(pw_rec_pe_abs(Val(n), "Abc") for n in 0:3)

((Val{true}(), ""), (Val{false}(), "Abc"), (Val{false}(), "AbcAbc"), (Val{false}(), "AbcAbcAbc"))

In [39]:
pw_rec_pe_opt(::Val{n}, x::T) where {n,T} = pw_rec_pe_abs(Val(n), x)[2]

pw_rec_pe_opt (generic function with 1 method)

In [42]:
Tuple(pw_rec_pe_opt(Val(n), "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [44]:
@code_typed pw_rec_pe_opt(Val(5), 10)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.mul_int(%1, %1)::Int64
│   %3 = Base.mul_int(%2, x)::Int64
└──      return %3
) => Int64